In [1]:
import sys
import os
from pathlib import Path
current_dir = os.getcwd()  
project_root = os.path.dirname(current_dir)  
sys.path.insert(0, project_root) 
import numpy as np
import pandas as pd
from src.data.data_preprocessor import DataPreprocessor
from src.config.config import Config
data_preprocessor = DataPreprocessor()

def detect_and_remove_anomalies(df, target_col, country_col='Country Name', year_col='Year', 
                               z_threshold=2.0, ratio_threshold=2.0, 
                               min_years=3, return_anomalies=False):
    """
    检测并移除目标列中的异常值
    
    参数:
    - df: 数据框
    - target_col: 目标列名（建筑垃圾量）
    - country_col: 国家列名
    - year_col: 年份列名
    - z_threshold: Z-score阈值，超过此值视为异常
    - ratio_threshold: 比值阈值，与相邻年份比值超过此值视为异常
    - min_years: 最少需要的年份数据量，少于此值的国家不进行异常检测
    - return_anomalies: 是否返回异常值信息
    
    返回:
    - 清洗后的数据框，如果return_anomalies=True，则同时返回异常值信息
    """
    # 复制数据框，避免修改原始数据
    df_clean = df.copy()
    
    # 存储所有异常记录
    all_anomalies = pd.DataFrame()
    
    # 按国家分组处理
    for country, country_data in df_clean.groupby(country_col):
        # 如果数据量太少，跳过异常检测
        if len(country_data) < min_years:
            continue
            
        # 按年份排序
        country_data = country_data.sort_values(year_col)
        
        # 方法1: 基于Z-score的异常检测
        z_scores = np.abs((country_data[target_col] - country_data[target_col].mean()) / country_data[target_col].std())
        z_anomalies = country_data[z_scores > z_threshold].index
        
        # 方法2: 基于相邻年份比值的异常检测
        country_data_sorted = country_data.sort_values(year_col)
        
        # 计算与前一年的比值
        prev_ratios = country_data_sorted[target_col] / country_data_sorted[target_col].shift(1)
        # 计算与后一年的比值
        next_ratios = country_data_sorted[target_col] / country_data_sorted[target_col].shift(-1)
        
        # 找出异常记录
        ratio_anomalies = country_data_sorted[
            ((prev_ratios > ratio_threshold) | (prev_ratios < 1/ratio_threshold)) & 
            ((next_ratios > ratio_threshold) | (next_ratios < 1/ratio_threshold))
        ].index
        
        # 方法3: 考虑年份间隔的异常检测
        # 计算年份间隔
        year_gaps = country_data_sorted[year_col].diff()
        # 调整比值阈值，根据年份间隔
        adjusted_ratios = prev_ratios / year_gaps
        gap_anomalies = country_data_sorted[
            (adjusted_ratios > ratio_threshold) & (year_gaps > 1)
        ].index
        
        # 综合多种方法，确定最终的异常值
        # 至少满足两种方法判定为异常的记录
        anomaly_indices = list(set(z_anomalies) | set(ratio_anomalies) | set(gap_anomalies))
        
        if anomaly_indices and return_anomalies:
            # 收集异常记录信息
            anomalies = country_data.loc[anomaly_indices].copy()
            anomalies['z_score'] = z_scores.loc[anomaly_indices]
            anomalies['prev_ratio'] = prev_ratios.loc[anomaly_indices]
            anomalies['next_ratio'] = next_ratios.loc[anomaly_indices]
            anomalies['year_gap'] = year_gaps.loc[anomaly_indices]
            all_anomalies = pd.concat([all_anomalies, anomalies])
        
        # 从清洗数据中移除异常值
        df_clean = df_clean.drop(anomaly_indices)
    
    if return_anomalies:
        return df_clean, all_anomalies
    else:
        return df_clean

In [2]:
"""加载1990-2022数据"""
historical_df = pd.read_excel(
    Config.FEATURE_CONFIG['historical_data_path'],
    sheet_name=Config.FEATURE_CONFIG['historical_sheet'],
    usecols=Config.FEATURE_CONFIG['usecols'] 
)

"""执行完整处理流程"""
# 第一阶段：处理历史数据
all_countries_df = data_preprocessor.process_historical_data(historical_df)
features_path = Path(Config.PATH_CONFIG['features_dir']) / 'global_features.csv'
all_countries_df.to_csv(features_path, index=False)

In [3]:
# 第二阶段：处理MSW数据
"""加载包含MSW的目标数据"""
msw_df = pd.read_excel(
    Config.FEATURE_CONFIG['historical_msw_data_path'],
    sheet_name=Config.FEATURE_CONFIG['historical_msw_sheet'],
    usecols=Config.FEATURE_CONFIG['usecols'] + [Config.DATA_CONFIG['target_column']]
)

# 先检测异常值但不移除，查看异常值情况
target_col = Config.DATA_CONFIG['target_column']
_, anomalies = detect_and_remove_anomalies(msw_df, target_col, return_anomalies=True)
print(f"检测到 {len(anomalies)} 条异常记录:")
print(f"检测到 {len(anomalies)} 条异常记录:")
if len(anomalies) > 0:
    display(anomalies.sort_values(['Country Name', 'Year']))
else:
    print("没有检测到异常记录")

# 分析每个国家数据的分布情况
print("\n===== 各国数据分布情况分析 =====")
for country, country_data in msw_df.groupby('Country Name'):
    if len(country_data) < 3:  # 跳过数据量太少的国家
        continue
        
    country_data = country_data.sort_values('Year')
    
    # 计算Z-score
    z_scores = np.abs((country_data[target_col] - country_data[target_col].mean()) / country_data[target_col].std())
    
    # 计算年度增长比例
    growth_ratios = country_data[target_col].pct_change()
    
    print(f"\n国家: {country}, 数据点数: {len(country_data)}")
    print(f"平均值: {country_data[target_col].mean():.2f}, 标准差: {country_data[target_col].std():.2f}")
    print(f"最大Z-score: {z_scores.max():.2f}")
    print(f"最大年度增长率: {growth_ratios.max():.2%}")
    print(f"最小年度增长率: {growth_ratios.min():.2%}")
    
    # 显示可能的异常点
    potential_anomalies = country_data[(z_scores > 2.0) | (growth_ratios > 0.5) | (growth_ratios < -0.3)]
    if len(potential_anomalies) > 0:
        print("潜在异常点:")
        display(potential_anomalies[['Year', target_col]])


检测到 131 条异常记录:
检测到 131 条异常记录:


,Year,Country Name,Population,GDP PPP 2017,GDP PPP/capita 2017,Urban population %,Region,Income Group,CW,z_score,prev_ratio,next_ratio,year_gap
20,1999,Albania,3108778,1.702018e+10,5474.88,41.169,Europe & Central Asia,Upper middle income,6.200685e+05,2.412861,NaN,0.779770,NaN
182,2009,Aruba,99212,3.634069e+09,36629.33,43.421,Latin America & Caribbean,High income,1.489156e+05,2.872538,1.326175,2.503745,1.0
183,2010,Aruba,100341,3.534733e+09,35227.21,43.059,Latin America & Caribbean,High income,5.947713e+04,2.618311,0.399402,0.607011,1.0
264,1995,Azerbaijan,7684850,2.287776e+10,2977.00,52.206,Europe & Central Asia,Upper middle income,2.071237e+05,1.522800,0.451317,0.278198,1.0
265,1996,Azerbaijan,7763000,2.317517e+10,2985.34,51.896,Europe & Central Asia,Upper middle income,7.445200e+05,0.934773,3.594567,3.190902,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4841,2022,United Arab Emirates,9441129,7.073074e+11,74917.67,87.543,Middle East & North Africa,High income,1.410635e+07,2.016407,1.062746,NaN,1.0
4940,1992,Uzbekistan,21449000,6.594836e+10,3074.66,42.308,Europe & Central Asia,Lower middle income,1.355550e+06,2.154742,0.337496,0.516833,1.0
4969,2021,Uzbekistan,34915100,2.723553e+11,7800.50,50.427,Europe & Central Asia,Lower middle income,6.290539e+06,2.118646,1.149347,0.960819,1.0
4970,2022,Uzbekistan,35648100,2.877946e+11,8073.21,50.466,Europe & Central Asia,Lower middle income,6.547060e+06,2.340778,1.040779,NaN,1.0



===== 各国数据分布情况分析 =====

国家: Afghanistan, 数据点数: 20
平均值: 2402640.60, 标准差: 538450.40
最大Z-score: 1.58
最大年度增长率: 19.45%
最小年度增长率: -23.12%

国家: Albania, 数据点数: 21
平均值: 1036434.87, 标准差: 172561.27
最大Z-score: 2.41
最大年度增长率: 28.24%
最小年度增长率: -0.54%
潜在异常点:


,Year,CW
20,1999,620068.4863



国家: Algeria, 数据点数: 33
平均值: 7302317.82, 标准差: 1270021.95
最大Z-score: 1.69
最大年度增长率: 13.04%
最小年度增长率: -10.77%

国家: Angola, 数据点数: 33
平均值: 5155324.79, 标准差: 1676727.26
最大Z-score: 1.50
最大年度增长率: 10.32%
最小年度增长率: -16.62%

国家: Argentina, 数据点数: 31
平均值: 14411618.23, 标准差: 3872458.93
最大Z-score: 1.83
最大年度增长率: 72.05%
最小年度增长率: -52.42%
潜在异常点:


...


国家: Uzbekistan, 数据点数: 33
平均值: 3843886.16, 标准差: 1154818.65
最大Z-score: 2.34
最大年度增长率: 93.49%
最小年度增长率: -66.25%
潜在异常点:


,Year,CW
4939,1991,4016494.148
4940,1992,1355549.721
4941,1993,2622800.457
4942,1994,5059048.153
4943,1995,3093861.380
4969,2021,6290538.627
4970,2022,6547060.255



国家: Vanuatu, 数据点数: 33
平均值: 8181.67, 标准差: 2241.03
最大Z-score: 1.67
最大年度增长率: 34.45%
最小年度增长率: -26.34%

国家: Zambia, 数据点数: 33
平均值: 1347503.74, 标准差: 457215.14
最大Z-score: 2.09
最大年度增长率: 17.15%
最小年度增长率: -7.42%
潜在异常点:


,Year,CW
5036,2022,2301240.609



国家: Zimbabwe, 数据点数: 33
平均值: 1178904.37, 标准差: 286584.90
最大Z-score: 1.98
最大年度增长率: 17.20%
最小年度增长率: -20.59%


In [4]:
df_clean = detect_and_remove_anomalies(msw_df, target_col)
print(f"原始数据: {len(msw_df)} 行, 清洗后: {len(df_clean)} 行")

原始数据: 5070 行, 清洗后: 4939 行


In [5]:
train_df, predict_df = data_preprocessor.merge_features(df_clean)
print(f"用于训练的数据: {len(train_df)} 行, 预测的历史数据: {len(predict_df)} 行,总共: {len(train_df)+len(predict_df)}行")

用于训练的数据: 4939 行, 预测的历史数据: 492 行,总共: 5431行


In [6]:
# 保存最终数据集
train_df.to_csv(
    Path(Config.PATH_CONFIG['features_dir']) / 'training_data.csv', 
    index=False,
    encoding='utf-8-sig'
)
predict_df.to_csv(
    Path(Config.PATH_CONFIG['features_dir']) / 'prediction_data.csv', 
    index=False,
    encoding='utf-8-sig'
)

In [7]:
# """加载2023-2050数据"""
# future_df = pd.read_excel(
#     Config.FEATURE_CONFIG['future_data_path'],
#     sheet_name=Config.FEATURE_CONFIG['future_sheet'],
#     usecols=Config.FEATURE_CONFIG['usecols']
# )

# """加载1990-2022数据"""
# historical_df = pd.read_excel(
#     Config.FEATURE_CONFIG['historical_data_path'],
#     sheet_name=Config.FEATURE_CONFIG['historical_sheet'],
#     usecols=Config.FEATURE_CONFIG['usecols']
# )
# future_prediction_df = data_preprocessor.process_future_data(historical_df, future_df)

# # 保存最终数据集
# future_prediction_df.to_csv(
#     Path(Config.PATH_CONFIG['features_dir']) / 'future_prediction_data.csv', 
#     index=False,
#     encoding='utf-8-sig'
# )